<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/Whisper_return_timestamp_bug_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 10.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Just for future reference, this test was we are on `transformers 4.26.1` at the time of testing.

In [12]:
!transformers-cli env

2023-02-12 17:23:06.850265: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-12 17:23:06.850464: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-12 17:23:06.850495: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Instructions for updating:
Use

In [2]:
from transformers import pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small", generate_kwargs={"task": "transcribe", "language": "german"})

Loading the CV11 german split in streaming mode

In [4]:
from datasets import load_dataset
cv11 = load_dataset("mozilla-foundation/common_voice_11_0", "de", streaming=True, split="test")

Checking the first element to see if the transcription actually is `german`

In [5]:
next(iter(cv11))

Reading metadata...: 16082it [00:01, 13506.53it/s]


{'client_id': '0052c07533a6976233ad5926d950b523002c4d8cdd9ae8726dbfec385951bd22aa707a742c49afe20c7d6cb9515dbaddac5b4d6fe8ebddcfbec46a2d3180a3a1',
 'path': 'common_voice_de_17922420.mp3',
 'audio': {'path': 'common_voice_de_17922420.mp3',
  'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -9.0749630e-12,  5.6385865e-09,  7.3282314e-09], dtype=float32),
  'sampling_rate': 48000},
 'sentence': 'Zieht euch bitte draußen die Schuhe aus.',
 'up_votes': 2,
 'down_votes': 0,
 'age': '',
 'gender': '',
 'accent': '',
 'locale': 'de',
 'segment': ''}

Looks pretty german to me. In case you missed it, look at the `sentence` key.

Now, let's create a test sample from this sample.

In [6]:
test_sample = {"raw": next(iter(cv11))["audio"]["array"],
               "sampling_rate": next(iter(cv11))["audio"]["sampling_rate"]}

Reading metadata...: 16082it [00:00, 25586.44it/s]
Reading metadata...: 16082it [00:00, 47786.81it/s]


In [7]:
pipe(test_sample, chunk_length_s=30)

ValueError: ignored

As seen above the pipeline simply refuses to accept the defined `generate_kwargs`

Let's try with `return_timestamps=True` and see how the response to this is.

In [10]:
test_sample = {"raw": next(iter(cv11))["audio"]["array"],
               "sampling_rate": next(iter(cv11))["audio"]["sampling_rate"]}

Reading metadata...: 16082it [00:00, 32917.74it/s]
Reading metadata...: 16082it [00:00, 43461.71it/s]


In [11]:
pipe(test_sample, return_timestamps=True, chunk_length_s=30, stride_length_s=[6,0])

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': ' Please take off your shoes.',
 'chunks': [{'text': ' Please take off your shoes.', 'timestamp': (0.0, 3.0)}]}

Now, the pipeline works, however, completely refuses to accept the `generate_kwargs` i.e. doesn't transcribe in german.

Attempt 2: Let's remove the `generate_kwargs` and try with `forced_decoder_ids`.

In [13]:
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

In [14]:
pipe.model.config.forced_decoder_ids = (
    pipe.tokenizer.get_decoder_prompt_ids(
        language="de", task="transcribe"
    )
)

In [15]:
test_sample = {"raw": next(iter(cv11))["audio"]["array"],
               "sampling_rate": next(iter(cv11))["audio"]["sampling_rate"]}

Reading metadata...: 16082it [00:00, 55065.31it/s]
Reading metadata...: 16082it [00:00, 30277.58it/s]


In [16]:
pipe(test_sample, chunk_length_s=30)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': ' Zieht euch bitte draußen die Schuhe aus.'}

Okay, that seem to work perfectly. Let's try and use the same with `return_timestamps=True`

In [18]:
test_sample = {"raw": next(iter(cv11))["audio"]["array"],
               "sampling_rate": next(iter(cv11))["audio"]["sampling_rate"]}

Reading metadata...: 16082it [00:00, 38814.65it/s]
Reading metadata...: 16082it [00:00, 46545.52it/s]


In [19]:
pipe(test_sample, return_timestamps=True, chunk_length_s=30, stride_length_s=[6,0])

{'text': ' Please take off your shoes.',
 'chunks': [{'text': ' Please take off your shoes.', 'timestamp': (0.0, 3.0)}]}

Um, again, `translates` to english.